In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

Join together UW orgs with the CRA/211 data

In [2]:
df_orgs = pd.read_csv('../joined-data/simplified_matches.csv')

df_uw = pd.read_excel('../cra-data/cre-uw-checking 2.xlsx', sheet_name="UW Orgs")
df_uw = df_uw.rename(columns={'CHARITABLE REGISTRATION NUMBER (BN)': 'CRA_BN_ID'})

print(f'Total UW orgs: {len(df_uw)}')

# df_uw = df_uw[df_uw['CRA_BN_ID'].notna()]
# print(f'Total UW orgs (with CRA BN ID): {len(df_uw)}')

num_uw_matches = len(set(df_uw['CRA_BN_ID']).intersection(set(df_orgs['CRA_BN_ID'])))
print(f'No. UW orgs with 211/CRA data: {num_uw_matches}')

Total UW orgs: 278
No. UW orgs with 211/CRA data: 187


In [3]:
df_uw_cra211 = pd.merge(
    left = df_uw,
    right = df_orgs,
    on = 'CRA_BN_ID',
    how = 'left'
)
df_uw_cra211 = df_uw_cra211.sort_values(by='CRA_Organization_Name')
df_uw_cra211.to_csv('../joined-data/uw_cra211.csv', index=False)

Join tenure data

In [23]:
df_tenure_by_cra = pd.read_excel('../../2024-work/SPRE_data/2021_CRE_DATA_Oct9.xlsx')
df_tenure_by_cra = df_tenure_by_cra.drop_duplicates()
# ['CRA_BN', 'CRA_LegalName', '211 Parent Agency Name', '211_Address1', '211_PostalCode', 'Tenure', 'Latitude', 'Longitude']
df_tenure_by_cra = df_tenure_by_cra[['CRA_BN', 'Latitude', 'Longitude', 'Tenure']].rename(
    columns={
        'CRA_BN': 'CRA_BN_ID',
        # 'CRA_LegalName': 'CRA_Organization_Name',
        # '211 Parent Agency Name': '211_Organization_Name',
        # '211_Address1': '211_Address',
        # '211_PostalCode': '211_Postal_Code',
        'Latitude': 'Y_Coordinate',
        'Longitude': 'X_Coordinate',
    }
)

# df_tenure_by_cra

In [26]:
df_uw_cra211_tenure = pd.merge(
    left = df_uw_cra211,
    right = df_tenure_by_cra,
    on = ['CRA_BN_ID', 'Y_Coordinate', 'X_Coordinate'],
    how = 'left'
)
df_uw_cra211_tenure.to_csv('../joined-data/uw_cra211_tenure.csv', index=False)

num_tenure = df_uw_cra211_tenure['Tenure'].count()
num_locs = len(df_uw_cra211_tenure)
pct_tenure = round(num_tenure / num_locs, 4) * 100
print(f'Number of locations with tenure data: {num_tenure}')
print(f'...which is {pct_tenure}% of all locations')
print(f'...so {num_locs - num_tenure} locations are missing tenure data')
print(f'...as there are {num_locs} locations in total')

Number of locations with tenure data: 428
...which is 48.42% of all locations
...so 456 locations are missing tenure data
...as there are 884 locations in total
